In [13]:
from PIL import Image
import numpy as np
import glob as glob
import pandas as pd
OFFSET = 104
LABELS = ["Meat", "Nuts/seeds", "Eggs", "Beans/lentils/peas", "Fruit", "Grain", "Vegetables", "Dairy", "Sauce/Spread", "Soup/Drink"]
LABELS_MAP = {}
for i in range(len(LABELS)):
    LABELS_MAP[LABELS[i]] = i+OFFSET+1
labelids = pd.read_csv("Nutrition5kClassification/complete_labels.csv",sep=',')
print(labelids.iloc[1][LABELS].to_numpy().squeeze())



[0 0 0 0 0 0 0 0 0 0]


REMAP SEGMENTATION MASKS TO FOOD GROUP LABELS

In [14]:
splits = ["train", "test"]
for s in splits:        
    for im_path in glob.glob(f"FoodSeg103/Images/ann_dir/{s}/*.png"):
        img = Image.open(im_path).convert('L')
        arr = np.array(img.getdata(), dtype=np.uint8)
        unique = np.unique(arr)
        arr = np.resize(arr, (img.size[1], img.size[0]))
        filename = im_path.split("/")[-1]
        skip_file = False
        for ingred_id in unique:
            labels = labelids.iloc[ingred_id][LABELS].to_numpy().squeeze()
            nonzeros = np.nonzero(labels)[0]
            if(len(nonzeros) >1):
                skip_file = True
                break
            elif(len(nonzeros) ==0):
                if(ingred_id!=0):
                    skip_file = True
                    break
                arr[arr == ingred_id] = OFFSET
            else:
                arr[arr == ingred_id] = LABELS_MAP[LABELS[nonzeros[0]]]
        if(skip_file):
            continue
        arr = arr-OFFSET
        img = Image.fromarray(arr, mode='L')
        newPath = f"FoodSeg103/Images/ann_dir_edited/{s}/{filename}"
        img.save(newPath)

In [15]:
import shutil

for file in glob.glob("FoodSeg103/Images/ann_dir_edited/test/*.png"):
    file_name = file.split("/")[-1].split(".")[0]+ ".jpg"
    dest = "FoodSeg103/Images/img_dir_edited/test/" + file_name
    src = "FoodSeg103/Images/img_dir/test/" + file_name
    shutil.copyfile(src, dest)

In [16]:
for file in glob.glob("FoodSeg103/Images/ann_dir_edited/train/*.png"):
    file_name = file.split("/")[-1].split(".")[0]+ ".jpg"
    dest = "FoodSeg103/Images/img_dir_edited/train/" + file_name
    src = "FoodSeg103/Images/img_dir/train/" + file_name
    shutil.copyfile(src, dest)